# 3. Inception model

This notebook provides an example of building a neural net with one Inception module. The convnet is a digit classifier.

---

We first load the MNIST data

In [1]:
from keras.datasets import mnist

(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

Using TensorFlow backend.


We preprocess the image as previously

In [2]:
from utils import preprocess

train_images = preprocess(train_images)
test_images = preprocess(test_images)

We also need to one-hot encode the labels

In [3]:
from keras.utils import to_categorical

train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

We can construct our neural net with inception module. Keep in mind that when we concatenate different branches, different inputs should have matching shapes except for the concat axis

In [4]:
from keras import layers
from keras import Input
from keras.models import Model


inputs = Input(shape=(28, 28, 1))

# branch a
a = layers.Conv2D(64, (1, 1), activation='relu')(inputs)
a = layers.MaxPool2D(2, 2)(a)

# branch b
b = layers.Conv2D(32, (1, 1), activation='relu')(inputs)
b = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(b)
b = layers.MaxPool2D(2, 2)(b)

# branch c
c = layers.Conv2D(32, (1, 1), activation='relu')(inputs)
c = layers.Conv2D(64, (5, 5), activation='relu', padding='same')(c)
c = layers.MaxPool2D(2, 2)(c)

# branch d
d = layers.MaxPool2D(2, 2)(inputs)
d = layers.Conv2D(64, (1, 1), activation='relu')(d)

# concatenation
concat = layers.concatenate([a, b, c, d], axis=-1)

# add dense layers
output = layers.Flatten()(concat)
output = layers.Dense(32, activation='relu')(output)
output = layers.Dense(10, activation='softmax')(output)

Instantiate the model and compile the model

In [5]:
model = Model(inputs, output)
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

Train model

In [6]:
model.fit(train_images, train_labels, epochs=1, batch_size=64)

Epoch 1/1
60000/60000 [==============================] - 941s - loss: 0.1539 - acc: 0.9539   


Since we have multiple branches the computation is getting more intensive: 1 epoch is taking 10-15 mins on a modern laptop. This kind of heavy lifting calls for GPU.

But as long as the network is training then rest assured the Inception module is constructed properly!

In [7]:
test_loss, test_acc = model.evaluate(test_images, test_labels)

10000/10000 [==============================] - 41s    


In [8]:
test_acc

0.98219999999999996